# Битовое сжатие

* Из-за него в «асимптотиках» появляется `/64`
* На Всеросе часто дают задачи, где оно может принести «бесплатные» ~20 баллов
* `bitset` есть в stl; говорят, самописный быстрее

Процессор так устроен, что работает сразу с блоками по 32 или 64 бита (зависит от архитектуры, но получить что-то меньше одного байта он в принципе не может). Иными словами, сделать `&` двух `bool`-ом и двух `long`-ов примерно одинаково по скорости.

Часто нам требуется сделать много одинаковых операций над элементами булевого массива. Проксорить два массива, например. Здесь появляется такая идея: сгруппировать элементы массива в блоки по 64 и каждый блок считать двоичным числом. Тогда мы можем ксорить сразу все 64 бита за одну операцию.

Это всё можно кодить и вручную, но в STL это уже сделали до нас, создав структуру, ведущую себя как большое двоичное число со всеми стандартными битовыми операциями — `bitset`. 

Работать с ним нужно вот так:

In [ ]:
const int lim = 1000;
bitset<lim> b; // создать битсет размера lim (должно быть константой)
b.set();       // заполнить единицами
b.reset();     // заполнить нулями
b.flip();      // заменить единички на нули и наоборот
b.count();     // посчитать число единичек
cout << b;     // вывести битовую строку

Также для битсетов работает вся битовая арифметика — `&, |, ^, ~, <<, >>` и их варианты с `[operator]=`.

## Рюкзак

Задача: даны $n$ предметов с положительными целыми весами $a_i$ и рюкзак размера $lim$, выбрать подмножество предметов с максимальной суммой, не превышающий размер рюкзака.

Обычно его решают так:

In [ ]:
bool dp[lim] = {}; // так можно его заполнить нулями
dp[0] = 1;
for (int i = 0; i < n; i++)
    for (int x = lim - a[i]; x >= 0; x--)
        dp[x + a[i]] |= dp[x];

…а с битсетом оно разгоняется так:

In [ ]:
bitset<lim> b;
b[0] = 1;
for (int i = 0; i < n; i++)
    b |= b << a[i];

## Цикл длины 3

Пусть нам нужно узнать, есть ли цикл длины 3 в графе из $n$ вершин, заданном своей матрицей смежности. Обернем матрицу в битсет, и тогда задача решается за $O(\frac{n^3}{64})$ следующим образом:

In [ ]:
bitset<maxn> g[maxn]; // матрица смежности
for (int a = 0; a < n; a++) {
    for (int b = 0; b < n; b++) {
        if (g[a][b] && (g[a] & g[b]).any()) {
            // цикл найден
        }
    }
}

Benchmark: на серверах CodeForces этот код при $n = 5000$ работает за 7 секунд.

## Перемножение матриц

Матрица смежности графа, возведенная в степень $n$, имеет комбинаторный смысл: количество способов дойти из $a$ в $b$, используя ровно $n$ переходов. Иногда нам не нужно знать число способов, и нам просто хватит знания, можно ли вообще через $n$ ходов там оказаться. Тогда вместо числового умножения нам хватит битового умножения:

In [ ]:
typedef bitset<maxn> t;
typedef array<t, maxn> matrix;

matrix operator* (matrix a, matrix b) {
    matrix c;
    for(int i = 0; i < maxn; i++)
        for(int j = 0; j < maxn; j++)
            if(a[i][j])
                c[i] |= b[j];
    return c;
}

(Я не уверен на 100%, что код рабочий.)

## Гаусс

TODO